In [1]:
!pip install schedule
!pip install snowflake-connector-python

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import snowflake.connector
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import schedule
import time
from datetime import datetime, timezone

# Snowflake credentials
username = 'Harshiteggoz'
password = 'Eggoz@123'
account = 'wt08714.ap-south-1.aws'
warehouse = 'COMPUTE_WH'
database = 'EGGOZDB'
schema = 'MAPLEMONK'

# Connecting to Snowflake
conn = snowflake.connector.connect(
    user=username,
    password=password,
    account=account,
    warehouse=warehouse,
    database=database,
    schema=schema
)

# Email credentials
sender_email = 'abhishektiwari.nssc@gmail.com'
app_password = 'rpax rxpp mpws wzvb'
excel_file_path = 'transformed_data.xlsx'  # Use the same file for all recipients

# Read emails and names from CSV
emails_df = pd.read_csv('C:\\Users\\User\\Downloads\\Eggoz\\Data_Sales\\emails.csv')
print(emails_df.columns)
def transform(query):
    # Execute Snowflake SQL Queries
    # Connecting to Snowflake
    conn = snowflake.connector.connect(
        user=username,
        password=password,
        account=account,
        warehouse=warehouse,
        database=database,
        schema=schema
    )
    cur = conn.cursor()
    cur.execute(f"USE {database}.{schema}")

    result_df = pd.read_sql_query(query, conn)



    return result_df

output_excel_file = "transformed_data_output.xlsx"

def send_email(name, email, excel_file_path):
    # Create Excel attachment
    with open(excel_file_path, 'rb') as file:
        excel_attachment = MIMEText(file.read(), 'base64', 'utf-8')
        excel_attachment.add_header('Content-Disposition', 'attachment', filename='transformed_data_output.xlsx')

    # Create email message
    email_message = MIMEMultipart()
    email_message['From'] = sender_email
    email_message['To'] = email
    email_message['Subject'] = 'Transformed Data'
    email_message.attach(excel_attachment)

    # Customize email body
    email_body = f"Hi {name},\nPlease find attached the transformed dataset\nThanks"
    email_message.attach(MIMEText(email_body, 'plain'))

    # Send email using SMTP
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(sender_email, app_password)
        server.sendmail(sender_email, email, email_message.as_string())

def fetch_transform_send():
    # Define queries for each sheet with specific columns
    queries = [
        """
        SELECT QUANTITY, SKU_ORDER_PRICE_AMOUNT
        FROM eggozdb.maplemonk.darjan_sales_return_replace_promo_primary_and_secondary
        """,
        """
        SELECT ID, NAME
        FROM eggozdb.maplemonk.my_sql_product_product
        """,
        """
        SELECT ID, CODE
        FROM eggozdb.maplemonk.my_sql_retailer_retailer
        """
    ]

    # Iterate over queries and send emails
    for sheet_index, query in enumerate(queries, start=1):
        result_df = transform(query)
        print(result_df.columns)
        # Save DataFrame to the current sheet
        excel_sheet_name = f'Sheet{sheet_index}'

        # Check if the sheet already exists
        if excel_sheet_name in writer.sheets:
            # If the sheet exists, you can choose to overwrite or use a different name
            excel_sheet_name += "_new"  # Use a different name (you can customize this logic)

        excel_file_path = output_excel_file  # Store the Excel file path for sending email
        with pd.ExcelWriter(output_excel_file, engine='openpyxl', mode='a') as writer:
            result_df.to_excel(writer, sheet_name=excel_sheet_name, index=False)

            # Send customized emails to each recipient
            for index, row in emails_df.iterrows():
                send_email(row['name'], row['email'], excel_file_path)

    # Close the connection to Snowflake
    conn.close()

# Schedule the message to be sent every hour
schedule.every().minute.do(fetch_transform_send)




Index(['name', 'email'], dtype='object')


Every 1 minute do fetch_transform_send() (last run: [never], next run: 2023-12-22 14:23:50)

In [1]:
import snowflake.connector
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import schedule
import time

# Snowflake credentials
username = 'Harshiteggoz'
password = 'Eggoz@123'
account = 'wt08714.ap-south-1.aws'
warehouse = 'COMPUTE_WH'
database = 'EGGOZDB'
schema = 'MAPLEMONK'

# Connecting to Snowflake
conn = snowflake.connector.connect(
    user=username,
    password=password,
    account=account,
    warehouse=warehouse,
    database=database,
    schema=schema
)

# Email credentials
sender_email = 'abhishektiwari.nssc@gmail.com'
app_password = 'rpax rxpp mpws wzvb'
output_excel_file = 'transformed_data_output.xlsx'  # Use the same file for all recipients

# Read emails and names from CSV
emails_df = pd.read_csv('C:\\Users\\User\\Downloads\\Eggoz\\Data_Sales\\emails.csv')
print(emails_df.columns)

# Open the Excel file in 'a' (append) mode using openpyxl engine
with pd.ExcelWriter(output_excel_file, engine='openpyxl', mode='a') as writer:
    def transform(query):
        # Execute Snowflake SQL Queries
        # Connecting to Snowflake
        conn = snowflake.connector.connect(
            user=username,
            password=password,
            account=account,
            warehouse=warehouse,
            database=database,
            schema=schema
        )
        cur = conn.cursor()
        cur.execute(f"USE {database}.{schema}")

        result_df = pd.read_sql_query(query, conn)
            # Identify date columns and convert them to date without timezone
        date_columns = [col for col in result_df.columns if result_df[col].dtype == 'datetime64[ns, UTC]']

        for date_column in date_columns:
        # Check if the date column exists before attempting to convert
           if date_column in result_df.columns:
               result_df[date_column] = result_df[date_column].dt.date

        return result_df

    def send_email(name, email):
        # Create Excel attachment
        with open(output_excel_file, 'rb') as file:
            excel_attachment = MIMEText(file.read(), 'base64', 'utf-8')
            excel_attachment.add_header('Content-Disposition', 'attachment', filename='transformed_data_output.xlsx')

        # Create email message
        email_message = MIMEMultipart()
        email_message['From'] = sender_email
        email_message['To'] = email
        email_message['Subject'] = 'Transformed Data'
        email_message.attach(excel_attachment)

        # Customize email body
        email_body = f"Hi {name},\nPlease find attached the transformed dataset\nThanks"
        email_message.attach(MIMEText(email_body, 'plain'))

        # Send email using SMTP
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(sender_email, app_password)
            server.sendmail(sender_email, email, email_message.as_string())

    def fetch_transform_send():
        # Define queries for each sheet with specific columns
        queries = [
            """
            SELECT DATE, SKU_ORDER_PRICE_AMOUNT
            FROM eggozdb.maplemonk.darjan_sales_return_replace_promo_primary_and_secondary
            """,
            """
            SELECT ID, NAME
            FROM eggozdb.maplemonk.my_sql_product_product
            """,
            """
            SELECT ID, CODE
            FROM eggozdb.maplemonk.my_sql_retailer_retailer
            """
        ]

        # Iterate over queries and send emails
        for sheet_index, query in enumerate(queries, start=1):
            result_df = transform(query)

            # Save DataFrame to the current sheet
            excel_sheet_name = f'Sheet{sheet_index}'

            # Check if the sheet already exists
            if excel_sheet_name in writer.sheets:
                # If the sheet exists, you can choose to overwrite or use a different name
                excel_sheet_name += "_new"  # Use a different name (you can customize this logic)

            result_df.to_excel(writer, sheet_name=excel_sheet_name, index=False)

            # Send customized emails to each recipient for the current sheet
            for index, row in emails_df.iterrows():
                send_email(row['name'], row['email'])

        # Close the connection to Snowflake
        conn.close()

# Schedule the message to be sent every hour
schedule.every().minute.do(fetch_transform_send)

Index(['name', 'email'], dtype='object')


Every 1 minute do fetch_transform_send() (last run: [never], next run: 2023-12-22 14:53:08)

In [2]:
while True:
    schedule.run_pending()
    time.sleep(1)

C:\Users\User\AppData\Local\Temp\ipykernel_7880\3305108932.py:52: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql_query(query, conn)
